In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import normalize
import matplotlib.pyplot as plt
import pickle


In [2]:
def loadDataFiles():
    market_df = pickle.load(open('Market_train',"rb"))
    news_df = pickle.load(open("News_train", "rb"))
    print('Finished loading datafiles!')
    return market_df, news_df


In [3]:
def preprocess_data(mkt_df, news_df, mktdropfeatures, newsdropfeatures):
    mkt_df['time'] = pd.to_datetime(mkt_df['time'])
    news_df['time'] = pd.to_datetime(news_df['time'])
    mkt_df['time'] = mkt_df['time'].dt.date
    news_df['time'] = news_df['time'].dt.date
    assetCodes = []
    index = 0
    for x in news_df['assetCodes']:
        x = x.split(',')[0].split("'")[1]
        assetCodes.append(x)
    news_df['assetCode'] = np.asarray(assetCodes)
    irrelevantColumns = ['sourceTimestamp', 'firstCreated', 'sourceId', 
                         'headline', 'provider', 'subjects', 'audiences',
                        'headlineTag', 'marketCommentary', 'assetCodes', 'assetName']
    news_df.drop(irrelevantColumns, axis=1, inplace=True)
    news_df.drop(newsdropfeatures, axis=1, inplace=True)
    mkt_df.drop(['assetName'], axis=1, inplace=True)
    mkt_df.drop(mktdropfeatures, axis=1, inplace=True)
    modifiednews = news_df.groupby(['time','assetCode'], sort=False).aggregate(np.mean).reset_index()
    # join news reports to market data, note many assets will have many days without news data
    merged = pd.merge(mkt_df, modifiednews, how='left', on=['time', 'assetCode'], copy=False) 
    merged = merged.fillna(0)
    print('Finished preprocessing data!')
    return merged


In [4]:
market_data, news_data = loadDataFiles()


Finished loading datafiles!


In [5]:
mkt_drop_features = ['universe', 'returnsClosePrevRaw1', 
                     'returnsOpenPrevRaw1', 'returnsClosePrevRaw10',
                    'returnsOpenPrevRaw10']
news_drop_features = ['noveltyCount3D', 'noveltyCount5D', 'noveltyCount7D', 'volumeCounts3D', 'volumeCounts5D', 'volumeCounts7D']
X = preprocess_data(market_data, news_data, [], [])


Finished preprocessing data!


In [6]:
def normalizeY(ydf):
    ydf = (ydf + 1) / 2
    return ydf


In [7]:
X = X[X['returnsOpenNextMktres10'] >=-1]
X = X[X['returnsOpenNextMktres10'] <=1]

y = X['returnsOpenNextMktres10']
y = normalizeY(y)

X.drop(['returnsOpenNextMktres10'], axis=1, inplace=True)
assetCodesAndTime = X.iloc[:, :2]
X = X.iloc[:, 2:]

In [8]:
def regularize(df):
    for column in df:
        colmin = np.amin(df[column])
        colmax = np.amax(df[column])
        df[column] = (df[column] - colmin) / (colmax - colmin)
    return df


In [9]:
def getNNModel(numhiddenlayers, nodes, numfeatures): # returns NN given hidden layers and nodes
    layers = []
    layers.append(keras.layers.Flatten(input_shape=(numfeatures,)))

    for x in range(numhiddenlayers):
        layers.append(keras.layers.Dense(nodes, activation=tf.nn.relu, use_bias=True))

    layers.append(keras.layers.Dense(1, activation=tf.nn.sigmoid))
    model = keras.Sequential(layers)
    sgd = keras.optimizers.SGD(lr=.1)
    model.compile(optimizer=sgd,
              loss='mean_squared_error',
              metrics=['accuracy'])
    return model


In [10]:
X = regularize(X)

In [11]:
nnmodel = getNNModel(3, 15, len(X.columns.values))
nnmodel.fit(X, y, epochs=1, verbose=1, batch_size=1000000)
nnpredictions = nnmodel.predict(X)


Epoch 1/1
4072203/4072203 [==============================] - 9s 2us/step - loss: 0.0021 - acc: 0.0000e+00
